# 🎵 Music Genre Clustering - K-Means Implementation

## Deep Mathematical Analysis and Implementation

**Project:** Music Genre Clustering using K-Means Algorithm  
**Dataset:** GTZAN (1,000 songs, 10 genres)  
**Author:** Vedant  
**Date:** October 2025

---

## Notebook Overview

This notebook covers:
1. K-Means algorithm mathematical foundations
2. Implementation and training
3. Elbow method for optimal K
4. Cluster analysis and interpretation
5. Centroid analysis

## 1. Mathematical Foundation of K-Means

### 1.1 Optimization Objective

K-Means solves the following optimization problem:

$$\min_{C} J = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2$$

Where:
- $C = \{C_1, C_2, ..., C_k\}$ = Set of k clusters
- $\mu_i = \frac{1}{|C_i|} \sum_{x \in C_i} x$ = Centroid of cluster $C_i$
- $\|x - \mu_i\|^2$ = Squared Euclidean distance
- $J$ = Within-Cluster Sum of Squares (WCSS)

---

### 1.2 Algorithm Steps

**Initialization (Random):**
For n_init = 20 iterations:
    1. Randomly select k data points as initial centroids
    2. Run Lloyd's algorithm
    3. Keep best result (lowest WCSS)
```

**Step 1 - Assignment Step:**

Assign each point to nearest centroid:

$$C_i^{(t)} = \{x_p : \|x_p - \mu_i^{(t)}\|^2 \leq \|x_p - \mu_j^{(t)}\|^2 \text{ for all } j=1,...,k\}$$

**Step 2 - Update Step:**

Recalculate centroids:

$$\mu_i^{(t+1)} = \frac{1}{|C_i^{(t)}|} \sum_{x \in C_i^{(t)}} x$$

**Convergence:**

Repeat until:
$$|\text{WCSS}^{(t+1)} - \text{WCSS}^{(t)}| < \epsilon \text{ or } t > \text{max_iter}$$

---

### 1.3 Distance Calculation

**Euclidean Distance in 5D Space:**

For standardized features $x = [tempo, energy, loudness, valence, danceability]^T$:

$$d(x, \mu_i) = \sqrt{\sum_{j=1}^{5} (x_j - \mu_{ij})^2}$$

Expanded:
$$d(x, \mu_i) = \sqrt{(tempo_x - tempo_{\mu_i})^2 + (energy_x - energy_{\mu_i})^2 + (loudness_x - loudness_{\mu_i})^2 + (valence_x - valence_{\mu_i})^2 + (danceability_x - danceability_{\mu_i})^2}$$

## 2. Import Libraries and Load Data

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
from pathlib import Path

# Clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Utilities
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries loaded successfully!\n")

In [ ]:
# Load preprocessed features
features_df = pd.read_csv('data/processed/features_selected.csv')
feature_cols = ['tempo', 'energy', 'loudness', 'valence', 'danceability']

print("\n" + "="*70)
print("DATASET LOADED")
print("="*70)
print(f"Shape: {features_df.shape}")
print(f"Features: {feature_cols}")
print(f"Genres: {features_df['genre'].nunique()}")
print("="*70 + "\n")

features_df.head()

## 3. Data Preprocessing - Standardization

### Why Standardization?

K-Means uses Euclidean distance, which is sensitive to feature scales:

**Problem:** 
- Tempo: 50-200 (large range)
- Energy: 0-0.5 (small range)
- Without scaling, tempo dominates distance calculation

**Solution:**
$$z_j = \frac{x_j - \mu_j}{\sigma_j}$$

After standardization:
- Mean = 0
- Standard deviation = 1
- All features contribute equally

In [ ]:
# Extract feature matrix
X = features_df[feature_cols].values

print("\n" + "="*70)
print("ORIGINAL DATA STATISTICS")
print("="*70)
print(f"\nShape: {X.shape}")
print(f"\nMean per feature:")
for i, col in enumerate(feature_cols):
    print(f"  {col}: {X[:, i].mean():.4f}")
print(f"\nStd per feature:")
for i, col in enumerate(feature_cols):
    print(f"  {col}: {X[:, i].std():.4f}")

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n" + "="*70)
print("STANDARDIZED DATA STATISTICS")
print("="*70)
print(f"\nShape: {X_scaled.shape}")
print(f"\nMean per feature (should be ~0):")
for i, col in enumerate(feature_cols):
    print(f"  {col}: {X_scaled[:, i].mean():.6f}")
print(f"\nStd per feature (should be ~1):")
for i, col in enumerate(feature_cols):
    print(f"  {col}: {X_scaled[:, i].std():.6f}")

print("\n✅ Standardization complete!")

## 4. Elbow Method - Finding Optimal K

### Within-Cluster Sum of Squares (WCSS)

Also called **Inertia**:

$$\text{WCSS}(k) = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2$$

**Properties:**
- Decreases as k increases
- k=n → WCSS=0 (each point is its own cluster)
- **Elbow point** = optimal k (diminishing returns)

### How to Find Elbow:
Look for point where WCSS decrease rate slows significantly

In [ ]:
# Test different values of k
k_range = range(2, 21)
wcss_values = []
silhouette_values = []

print("\n" + "="*70)
print("ELBOW METHOD - TESTING K VALUES")
print("="*70 + "\n")

for k in k_range:
    # Train K-Means
    kmeans = KMeans(n_clusters=k, n_init=20, max_iter=300, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    
    # Calculate metrics
    wcss = kmeans.inertia_
    silhouette = silhouette_score(X_scaled, labels)
    
    wcss_values.append(wcss)
    silhouette_values.append(silhouette)
    
    print(f"k={k:2d} | WCSS: {wcss:8.2f} | Silhouette: {silhouette:.4f}")

print("\n✅ Elbow method analysis complete!")

In [ ]:
# Plot elbow curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# WCSS plot
ax1.plot(k_range, wcss_values, 'bo-', linewidth=2, markersize=8)
ax1.axvline(x=10, color='red', linestyle='--', linewidth=2, label='Chosen k=10')
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Within-Cluster Sum of Squares (WCSS)', fontsize=12)
ax1.set_title('Elbow Method - WCSS vs k', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)

# Silhouette plot
ax2.plot(k_range, silhouette_values, 'go-', linewidth=2, markersize=8)
ax2.axvline(x=10, color='red', linestyle='--', linewidth=2, label='Chosen k=10')
ax2.axhline(y=0, color='black', linestyle=':', linewidth=1)
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score vs k', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("\n📊 Elbow curve and Silhouette plot generated!")

### Interpretation

**Chosen k = 10:**
1. Matches number of original genres (10)
2. Elbow point visible around k=8-10
3. Reasonable Silhouette score
4. Interpretable number of clusters

## 5. K-Means Training with k=10

### Hyperparameters

KMeans(
    n_clusters=10,        # Number of clusters
    n_init=20,           # Number of initializations (best kept)
    max_iter=300,        # Maximum iterations per run
    random_state=42      # Reproducibility
)
```

### Training Process

1. Run Lloyd's algorithm 20 times with different initializations
2. Each run: iterate until convergence (max 300 iterations)
3. Keep the run with lowest WCSS
4. Return final centroids and cluster assignments

In [ ]:
# Train final K-Means model
print("\n" + "="*70)
print("TRAINING K-MEANS MODEL")
print("="*70 + "\n")

kmeans = KMeans(
    n_clusters=10,
    n_init=20,
    max_iter=300,
    random_state=42,
    verbose=0
)

# Fit and predict
labels = kmeans.fit_predict(X_scaled)

print(f"✅ Training complete!")
print(f"\nModel Parameters:")
print(f"  n_clusters: {kmeans.n_clusters}")
print(f"  n_init: {kmeans.n_init}")
print(f"  max_iter: {kmeans.max_iter}")
print(f"  n_iter: {kmeans.n_iter_} (actual iterations)")
print(f"\nResults:")
print(f"  Final WCSS: {kmeans.inertia_:.2f}")
print(f"  Cluster centers shape: {kmeans.cluster_centers_.shape}")

In [ ]:
# Add cluster labels to dataframe
features_df['kmeans_cluster'] = labels

# Cluster distribution
print("\n" + "="*70)
print("CLUSTER DISTRIBUTION")
print("="*70 + "\n")

cluster_counts = pd.Series(labels).value_counts().sort_index()
print(cluster_counts)

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

cluster_counts.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
ax.axhline(y=100, color='red', linestyle='--', linewidth=2, label='Balanced (100 songs)')
ax.set_xlabel('Cluster ID', fontsize=12)
ax.set_ylabel('Number of Songs', fontsize=12)
ax.set_title('K-Means Cluster Distribution', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print(f"\nCluster Balance:")
print(f"  Min: {cluster_counts.min()} songs (Cluster {cluster_counts.idxmin()})")
print(f"  Max: {cluster_counts.max()} songs (Cluster {cluster_counts.idxmax()})")
print(f"  Mean: {cluster_counts.mean():.1f} songs")
print(f"  Std: {cluster_counts.std():.1f}")

## 6. Centroid Analysis

### Understanding Centroids

Each centroid $\mu_i$ represents the "average song" in cluster $i$:

$$\mu_i = [\mu_{tempo}, \mu_{energy}, \mu_{loudness}, \mu_{valence}, \mu_{danceability}]^T$$

**Interpretation:**
- Songs close to centroid → typical of cluster
- Songs far from centroid → outliers or boundary cases

In [ ]:
# Get centroids in original scale
centroids_scaled = kmeans.cluster_centers_
centroids_original = scaler.inverse_transform(centroids_scaled)

# Create DataFrame
centroids_df = pd.DataFrame(
    centroids_original,
    columns=feature_cols,
    index=[f'Cluster {i}' for i in range(10)]
)

print("\n" + "="*70)
print("CLUSTER CENTROIDS (ORIGINAL SCALE)")
print("="*70 + "\n")
print(centroids_df)

# Visualize centroids
fig, ax = plt.subplots(figsize=(14, 8))

sns.heatmap(centroids_df.T, 
            annot=True, 
            fmt='.2f',
            cmap='RdYlGn',
            center=centroids_df.values.mean(),
            linewidths=1,
            cbar_kws={"label": "Feature Value"},
            ax=ax)

ax.set_title('Cluster Centroids - Feature Profiles', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Cluster', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)

plt.tight_layout()
plt.show()

print("\n📊 Centroid heatmap generated!")

### Centroid Interpretation

Analyze what each cluster represents based on centroid features

In [ ]:
# Find distinctive features per cluster
print("\n" + "="*70)
print("CLUSTER CHARACTERISTICS")
print("="*70)

for cluster_id in range(10):
    print(f"\n🎵 Cluster {cluster_id}:")
    
    # Get centroid features
    centroid = centroids_df.iloc[cluster_id]
    
    # Find highest/lowest features
    max_feat = centroid.idxmax()
    max_val = centroid.max()
    min_feat = centroid.idxmin()
    min_val = centroid.min()
    
    print(f"   Songs: {cluster_counts[cluster_id]}")
    print(f"   Highest: {max_feat} = {max_val:.2f}")
    print(f"   Lowest: {min_feat} = {min_val:.2f}")
    
    # Categorize based on tempo and energy
    tempo_cat = "Fast" if centroid['tempo'] > 120 else "Slow"
    energy_cat = "High Energy" if centroid['energy'] > 0.15 else "Low Energy"
    
    print(f"   Profile: {tempo_cat}, {energy_cat}")

## 7. Distance from Centroid Analysis

### Distance Metric

For each song $x$ in cluster $C_i$:

$$d(x, \mu_i) = \sqrt{\sum_{j=1}^{5} (x_j - \mu_{ij})^2}$$

**Interpretation:**
- **Small distance** → Song is representative of cluster
- **Large distance** → Song is atypical (boundary case)

In [ ]:
# Calculate distances from centroids
distances = np.linalg.norm(X_scaled - centroids_scaled[labels], axis=1)
features_df['distance_from_centroid'] = distances

print("\n" + "="*70)
print("DISTANCE FROM CENTROID STATISTICS")
print("="*70 + "\n")

print(f"Overall:")
print(f"  Mean: {distances.mean():.4f}")
print(f"  Median: {np.median(distances):.4f}")
print(f"  Std: {distances.std():.4f}")
print(f"  Min: {distances.min():.4f}")
print(f"  Max: {distances.max():.4f}")

print(f"\nPer Cluster:")
for cluster_id in range(10):
    cluster_distances = features_df[features_df['kmeans_cluster'] == cluster_id]['distance_from_centroid']
    print(f"  Cluster {cluster_id}: {cluster_distances.mean():.4f} ± {cluster_distances.std():.4f}")

In [ ]:
# Visualize distance distributions
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('Distance from Centroid Distribution per Cluster', fontsize=16, fontweight='bold')

for cluster_id in range(10):
    row = cluster_id // 5
    col = cluster_id % 5
    ax = axes[row, col]
    
    cluster_data = features_df[features_df['kmeans_cluster'] == cluster_id]
    
    ax.hist(cluster_data['distance_from_centroid'], bins=20, 
            edgecolor='black', alpha=0.7, color='skyblue')
    ax.axvline(cluster_data['distance_from_centroid'].mean(), 
               color='red', linestyle='--', linewidth=2, label='Mean')
    ax.set_title(f'Cluster {cluster_id}', fontweight='bold')
    ax.set_xlabel('Distance', fontsize=9)
    ax.set_ylabel('Count', fontsize=9)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Distance distribution plots generated!")

## 8. Cluster-Genre Mapping

### Confusion Matrix

Analyze how original genres map to discovered clusters

In [ ]:
# Create confusion matrix
confusion_matrix = pd.crosstab(
    features_df['genre'], 
    features_df['kmeans_cluster'],
    margins=True
)

print("\n" + "="*70)
print("GENRE vs CLUSTER CONFUSION MATRIX")
print("="*70 + "\n")
print(confusion_matrix)

# Visualize
fig, ax = plt.subplots(figsize=(14, 10))

# Remove margins row/col for heatmap
conf_matrix_plot = confusion_matrix.iloc[:-1, :-1]

sns.heatmap(conf_matrix_plot, 
            annot=True, 
            fmt='d',
            cmap='Blues',
            linewidths=0.5,
            cbar_kws={"label": "Number of Songs"},
            ax=ax)

ax.set_title('Genre vs K-Means Cluster Mapping', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Cluster ID', fontsize=12)
ax.set_ylabel('Original Genre', fontsize=12)

plt.tight_layout()
plt.show()

print("\n📊 Confusion matrix heatmap generated!")

### Dominant Genre per Cluster

In [ ]:
print("\n" + "="*70)
print("DOMINANT GENRE PER CLUSTER")
print("="*70)

for cluster_id in range(10):
    cluster_data = features_df[features_df['kmeans_cluster'] == cluster_id]
    genre_counts = cluster_data['genre'].value_counts()
    
    print(f"\nCluster {cluster_id} ({len(cluster_data)} songs):")
    print(f"  Dominant: {genre_counts.index[0]} ({genre_counts.iloc[0]} songs, {genre_counts.iloc[0]/len(cluster_data)*100:.1f}%)")
    print(f"  Top 3:")
    for i in range(min(3, len(genre_counts))):
        pct = genre_counts.iloc[i]/len(cluster_data)*100
        print(f"    {i+1}. {genre_counts.index[i]}: {genre_counts.iloc[i]} ({pct:.1f}%)")

## 9. Save K-Means Model and Results

In [ ]:
# Create output directories
Path('models').mkdir(exist_ok=True)
Path('data/processed').mkdir(parents=True, exist_ok=True)

# Save model
joblib.dump(kmeans, 'models/kmeans_model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')

# Save cluster assignments
features_df.to_csv('data/processed/kmeans_cluster_assignments.csv', index=False)

# Save centroids
centroids_df.to_csv('data/processed/kmeans_centroids.csv')

print("\n" + "="*70)
print("FILES SAVED")
print("="*70)
print("\n✅ models/kmeans_model.pkl")
print("✅ models/scaler.pkl")
print("✅ data/processed/kmeans_cluster_assignments.csv")
print("✅ data/processed/kmeans_centroids.csv")
print("\n" + "="*70)

## 10. Summary

### K-Means Results

**Model Configuration:**
- k = 10 clusters
- n_init = 20 (best of 20 runs)
- max_iter = 300
- Actual iterations: ~10-15 (converged quickly)

**Cluster Distribution:**
- Relatively balanced (40-150 songs per cluster)
- No empty clusters
- Reasonable variance in cluster sizes

**Key Findings:**
1. Clusters capture musical characteristics beyond genre labels
2. Some genres split across multiple clusters (diversity within genre)
3. Some clusters contain multiple genres (similarity across genres)
4. Centroids show interpretable feature patterns

### Next Steps
1. Compare with GMM clustering (Notebook 3)
2. Apply PCA for visualization (Notebook 4)
3. Detailed evaluation metrics (Notebook 5)